<a href="https://colab.research.google.com/github/vforjj/AI-ML-Review/blob/main/3_ModelServing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

test_df = pd.read_csv('dataset_preprocessing.csv', index_col=0)
label = test_df["Survived"]
test_df = test_df.drop(["Survived"], axis=1)
test_df.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,0.271174,0.125,0.0,0.014151,0,0,1,0,1,0,0,0,1
1,0.472229,0.125,0.0,0.139136,1,0,0,1,0,1,0,0,0
2,0.321438,0.000,0.0,0.015469,0,0,1,1,0,0,0,0,1
3,0.434531,0.125,0.0,0.103644,1,0,0,1,0,0,0,0,1
4,0.434531,0.000,0.0,0.015713,0,0,1,0,1,0,0,0,1


### 1. Staging Model 로드 및 예측

In [ ]:
# To do
# load staging model

model_uri = "models:/First_model_titanic/Staging"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [ ]:
loaded_model

RandomForestClassifier(max_depth=10, max_features=5, n_estimators=150)

In [ ]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)

0.9236812570145904

### 2. Production Model 로드 및 예측

In [ ]:
# To do
# load production model
model_uri = "models:/First_model_titanic/Production"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [ ]:
loaded_model

RandomForestClassifier(max_depth=10, max_features=5, n_estimators=150)

In [ ]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)

# 기존의 production accuracy : 0.920314253647587

0.9236812570145904

In [ ]:
test_df.iloc[0].values

### 3. MLFlow 를 이용한 API Serving

In [ ]:
mlflow models serve -m ./mlartifacts/899358510632907849/d20dbd4fbf2a421eab08dadf49bc8e36/artifacts/model -p 5001 --no-conda


In [ ]:
import pandas as pd
import requests

# Define local host and endpoint url
host = "127.0.0.1"
url = f"http://{host}:5001/invocations"

# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": test_df[:10].to_dict(orient="split")}

# Score model
response = requests.post(url, json=json_data)
print(f"\nPyfunc 'predict_interval':\n${response.json()}")



Pyfunc 'predict_interval':
${'predictions': [0, 1, 1, 1, 0, 0, 0, 1, 1, 1]}


In [ ]:
# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": test_df[:20].to_dict(orient="split")}

# Score model
response = requests.post(url, json=json_data)


In [ ]:

print(f"\nPyfunc 'predict_interval':\n${response.json()}")


Pyfunc 'predict_interval':
${'predictions': [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]}


### 4. Cloud 배포

##### AWS Sagemaker
* mlflow sagemaker build-and-push-container
* mlflow sagemaker deploy {parameters}

##### AzureML
* mlflow azureml export -m {model' path} -o test-output